<a href="https://colab.research.google.com/github/LokeshSreenathJ/L-Layer-Deep-NeuralNetwork/blob/main/L_Layer_Deep_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn import preprocessing

In [ ]:
data = load_breast_cancer()
X = np.array(data["data"]).T
Y = np.array(np.matrix(list(data["target"])))
X.shape

(30, 569)

In [ ]:
#Normalize the data
scaler = preprocessing.StandardScaler().fit(X)
X_std = scaler.transform(X)


In [ ]:
layer_dims = [30,7,4,2,1]

In [ ]:
def initialize_parameters(layer_dims):
    parameters = {}
    for i in range(1,len(layer_dims)):
        #using Xavier initialization efficient using tanh activation.
        np.random.seed(10)
        parameters["W"+ str(i)] = np.random.randn(layer_dims[i],layer_dims[i-1])*np.sqrt(1/(layer_dims[i-1]))
        parameters["b"+ str(i)] = np.zeros((layer_dims[i],1))
    return parameters            

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
tanh(X)

C:\Users\ljakka2\AppData\Local\Temp\ipykernel_5308\896133741.py:2: RuntimeWarning: overflow encountered in exp
  return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))
C:\Users\ljakka2\AppData\Local\Temp\ipykernel_5308\896133741.py:2: RuntimeWarning: invalid value encountered in true_divide
  return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))


array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        0.99999964],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.25933936, 0.18388432, 0.23832737, ..., 0.14085718, 0.25896622,
        0.        ],
       [0.43016571, 0.26827118, 0.3463586 , ..., 0.21823301, 0.38736831,
        0.27946347],
       [0.11834285, 0.08878559, 0.08735676, ..., 0.07804099, 0.12336834,
        0.07027397]])

In [ ]:
def tanh(x):
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

In [ ]:
def relu(x):
    dG = (x>0).astype(int)
    x = np.array(dG)*np.array(x)
    return x    

In [ ]:
def get_layer_values(A_prev,W,b, activation):
    Z = np.dot(W,A_prev)+b
    if activation ==1 or activation== "sigmoid":
        A = sigmoid(Z)
    elif activation == 2 or activation == "tanh":
        A = tanh(Z)
    elif activation == 3 or activation == "relu":
        A = relu(Z)
    linear_cache = (A_prev,W,b)
    activation_cache = Z
    cache = (linear_cache, activation_cache)
    return A, cache

In [ ]:
def get_cost(Y,AL):
    m = Y.shape[1]
    cost = -(np.dot(np.log(AL),Y.T)+np.dot(np.log(1-AL),(1-Y).T))*(1/m)
    return cost

In [ ]:
def forward_propagation(X, parameters, activation):
    caches = []
    A_prev = X
    L = int(len(parameters)//2)
    for i in range(1,L+1):
        A,cache_temp = get_layer_values(A_prev= A_prev, W= parameters["W"+ str(i)], b = parameters["b"+ str(i)], activation = activation[i])
        caches.append(cache_temp)
        A_prev = A
    cost = get_cost(Y, AL = A)
    cost = np.squeeze(cost)
    AL = A
    return AL, caches, cost

In [ ]:
def get_dZ(dA,cache,activation):
    if activation == 1:
        n = 1/(1+np.exp(cache[1]))
        dG = n*(1-n)
        dZ = dA*dG
    elif activation == 2:
        dG = 1-np.square(cache[1])
        dZ = dA*dG
    else:
        dG = (cache[1]>0).astype(int)
        dZ = dA*dG
    return dZ     

In [ ]:
def backward_propagation(AL,Y, caches, activation):
    grads = {}
    dAL = -(np.divide(Y,AL)- np.divide(1-Y, 1-AL))
    L = len(caches)
    m = AL.shape[1]
    #Y = Y.reshape(AL.shape)
    for i in reversed(range(L)):
        grads["dW"+ str(i+1)] = np.dot(get_dZ(dA= dAL,cache = caches[i], activation = activation[i+1]),caches[i][0][0].T)/m
        grads["db"+ str(i+1)] = np.mean(get_dZ(dA = dAL,cache = caches[i], activation = activation[i+1]), axis = 1, keepdims = True)
        grads["dA"+ str(i)] = np.dot(caches[i][0][1].T,get_dZ(dA = dAL,cache = caches[i], activation = activation[i+1]))
        dAL = grads["dA"+ str(i)]
    return grads    

In [ ]:
def update_parameters(grads, parameters, learning_rate):
    updated_params = parameters.copy()
    L = len(updated_params)
    for i in range(1,((L//2)+1)):
        updated_params["W"+ str(i)] = parameters["W"+ str(i)] - learning_rate*grads["dW"+ str(i)]
        updated_params["b"+ str(i)] = parameters["b"+ str(i)] - learning_rate*grads["db"+ str(i)]
    return updated_params    

In [ ]:
def L_layer_model(X,Y,layers_dims, learning_rate ,num_iter , activation, print_cost = False):
    costs = []
    parameters = initialize_parameters(layers_dims)
    for i in range(num_iter):
        AL,caches, cost =forward_propagation(X,parameters, activation)
        grads = backward_propagation(AL, Y, caches, activation)
        parameters = update_parameters(grads, parameters, learning_rate)
        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0 or i == num_iter - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iter:
            costs.append(cost)
    return parameters, costs        
        


In [1]:
final_params, costs = L_layer_model(X_std,Y, layers_dims = [30,10,7,5,3,1], learning_rate = 0.06, num_iter = 2700, activation = [0,3,3,3,3,1], print_cost = True)

NameError: ignored

In [ ]:
a = np.array(np.matrix(X[:,0])).T
a.shape

(30, 1)

In [ ]:
def predict(X, final_params, activation):
    y = np.ones((1,X.shape[1]))
    y = X
    L = len(final_params)//2
    for i in range(1,L+1):
        A_prev = np.dot(final_params["W"+str(i)],y) + final_params["b"+ str(i)]
        if activation[i] == 1:
            A_prev = sigmoid(A_prev)
        elif activation[i] == 2:
            A_prev = tanh(A_prev)
        elif activation[i] == 3:
            A_prev = relu(A_prev)
        y = A_prev
        output = (y>0.5).astype(int)
    return output

In [ ]:
k = predict(X, final_params, activation = [0,3,3,3,3,1])

In [ ]:
count = 0
for i in range((X.shape[1])):
    if k[:,i]==Y[:,i]:
        count+=1
accuracy = count/X.shape[1]
accuracy

0.8804920913884007